# Mapping the 2021 La Palma *Cumbre Vieja* Eruption

In [1]:
import matplotlib.pyplot as plt
from ipywidgets import interact

import lplm_geoprocessing as lpgeo
import lplm_io as lpio
import lplm_presentation as lpp
import lplm_utils

# TODO: remove when not needed.
import xarray
import numpy

In [60]:
# TODO: remove when not needed
import importlib
importlib.reload(lpgeo)
importlib.reload(lpio)
importlib.reload(lpp)
importlib.reload(lplm_utils)

<module 'lplm_utils' from '/home/jovyan/lplm/lplm_utils.py'>

Start our journey...

Note: Mouse wheel scroll doesn't zoom - use double-click (zoom in) and shift & double-click (zoom-out) instead.

In [28]:
# Center needs to be passed in as EPSG:4326 regardless of desired map CRS. 
map_center = ( (28.58 + 28.65) / 2, (-17.97 + -17.84) / 2)
 
from sidecar import Sidecar
from ipyleaflet import Map, LayersControl, projections
lp_map = Map(center=map_center, zoom=13, crs=projections.EPSG3857)
lp_map.add_control(LayersControl())
sc = Sidecar()
with sc:
    display(lp_map)

## Load the image stack and choose a date

In [29]:
xds_grd = lpio.load_series("data")

In [30]:
plt.rcParams["figure.figsize"] = (12,8)
plt.rcParams["figure.dpi"] = 100

def view_scene(date):
    # Use this date for the rest of the notebook
    global selected_date 
    selected_date = date
    
    xds_grd.sel(date=date).plot(cmap="gray")
    
interact(view_scene, date=lpp.get_date_labels(xds_grd))

interactive(children=(Dropdown(description='date', options=(('2021-09-10', numpy.datetime64('2021-09-10T00:00:…

<function __main__.view_scene(date)>

Add this to the map.

In [61]:
from ipyleaflet import ImageOverlay
import base64

layer_sar = lpp.make_sar_layer(xds_grd, selected_date)
lp_map.add_layer(layer_sar)

## Segmentation

SLIC

In [36]:
lpgeo.apply_segmentation(xds_grd, selected_date)

In [37]:
gdf = lpgeo.vectorize_segments(xds_grd)

Display the result.

In [62]:
# This may take a little while.
from ipyleaflet import GeoData
# Have to convert to EPSG:4326 first, despite map CRS.
layer_seg = GeoData(
    geo_dataframe = gdf.to_crs("EPSG:4326"),
    style = { "fillOpacity": 0.0, "weight": 1.0 },
    name = "Segments")
lp_map.add_layer(layer_seg)

## Analysis, Part 1
Calculate statistical features of segments, and from those derive initial lava-likeness metrics.

In [75]:
xds_segstats = lpgeo.segment_stats(xds_grd)

In [76]:
lpgeo.enrich(gdf, xds_segstats, selected_date)

In [147]:
from ipyleaflet import Choropleth
from branca.colormap import linear
import json
# TODO: Map of lava likeness.
# lp_map.remove(layer_seg)

# Prepare geojson input for ipyleaflet's wacky choropleth interface.
# Thanks to Abdishakur, https://towardsdatascience.com/ipyleaflet-interactive-mapping-in-jupyter-notebook-994f19611e79
geojson = json.loads(gdf.loc[:, ["segment_id", "geometry"]].to_crs("EPSG:4326").to_json())
for feature in geojson["features"]:
    properties = feature["properties"]
    feature.update(id=properties["segment_id"])

layer_enriched = Choropleth(
    geo_data = geojson,
    choro_data = gdf["mean"].to_dict(),
    colormap=linear.viridis,
    style = { "fillOpacity": 0.5, "weight": 1.0 },
    name="Segment mean"
)
lp_map.add_layer(layer_enriched)


## Analysis, Part 2
Now apply some neighbourhood-based operations to estimate the lava field.

Choose a starting point - ideally somewhere clearly in the lava field.

In [45]:
from ipyleaflet import Marker
marker = Marker(location=map_center, draggable=True)
lp_map.add_layer(marker)